In [0]:
!pip install --upgrade tables
!pip install eli5
!pip.install xgboost
!pip install numpy==1.15.4

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [2]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix2/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix2/dw_matrix_car


In [3]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

### Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorizes_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorizes_values
  else:
    # przypisanie id do każdego koloru
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [5]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  # neg_mean_absolute_error = wyniki, które dostajemy, będą też miały wartość ujemną
  scores = cross_val_score(model, X, y, cv = 3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

### Decision Tree

In [13]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

### Random Forest

In [14]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

### XGBoost

In [16]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[16:22:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:22:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:23:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [17]:
m = xgb.XGBRegressor(max_depth = 5, n_estimators = 5, learning_rate = 0.1, seed = 0)
m.fit(X, y)

imp = PermutationImportance(m, random_state = 0).fit(X, y)
eli5.show_weights(imp, feature_names = cat_feats)

[16:25:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1554 ± 0.0014,param_napęd__cat
0.0957 ± 0.0020,param_skrzynia-biegów__cat
0.0884 ± 0.0018,param_faktura-vat__cat
0.0633 ± 0.0020,param_rok-produkcji__cat
0.0425 ± 0.0009,param_stan__cat
0.0387 ± 0.0010,feature_kamera-cofania__cat
0.0318 ± 0.0014,param_moc__cat
0.0167 ± 0.0010,param_pojemność-skokowa__cat
0.0104 ± 0.0001,feature_czujniki-parkowania-przednie__cat
0.0100 ± 0.0003,feature_regulowane-zawieszenie__cat


In [21]:
feats = ["param_napęd__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_rok-produkcji__cat","param_stan__cat","feature_kamera-cofania__cat","param_moc__cat","param_pojemność-skokowa__cat","feature_czujniki-parkowania-przednie__cat","feature_regulowane-zawieszenie__cat","feature_wspomaganie-kierownicy__cat","feature_asystent-pasa-ruchu__cat","feature_światła-led__cat","feature_łopatki-zmiany-biegów__cat","seller_name__cat","feature_bluetooth__cat","param_marka-pojazdu__cat","feature_system-start-stop__cat","feature_światła-do-jazdy-dziennej__cat","param_kraj-pochodzenia__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:34:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:34:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:34:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13768.627183898345, 148.0764583063271)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [27]:
feats = ["param_napęd__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_rok-produkcji","param_stan__cat","feature_kamera-cofania__cat","param_moc__cat","param_pojemność-skokowa__cat","feature_czujniki-parkowania-przednie__cat","feature_regulowane-zawieszenie__cat","feature_wspomaganie-kierownicy__cat","feature_asystent-pasa-ruchu__cat","feature_światła-led__cat","feature_łopatki-zmiany-biegów__cat","seller_name__cat","feature_bluetooth__cat","param_marka-pojazdu__cat","feature_system-start-stop__cat","feature_światła-do-jazdy-dziennej__cat","param_kraj-pochodzenia__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:37:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:37:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12060.795220210457, 151.4301935510006)

In [0]:
df['param_moc'].unique()

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ["param_napęd__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_rok-produkcji","param_stan__cat","feature_kamera-cofania__cat","param_moc","param_pojemność-skokowa","feature_czujniki-parkowania-przednie__cat","feature_regulowane-zawieszenie__cat","feature_wspomaganie-kierownicy__cat","feature_asystent-pasa-ruchu__cat","feature_światła-led__cat","feature_łopatki-zmiany-biegów__cat","seller_name__cat","feature_bluetooth__cat","param_marka-pojazdu__cat","feature_system-start-stop__cat","feature_światła-do-jazdy-dziennej__cat","param_kraj-pochodzenia__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

In [36]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa__cat'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')))
feats = ["param_napęd__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_rok-produkcji","param_stan__cat","feature_kamera-cofania__cat","param_moc","param_pojemność-skokowa","feature_czujniki-parkowania-przednie__cat","feature_regulowane-zawieszenie__cat","feature_wspomaganie-kierownicy__cat","feature_asystent-pasa-ruchu__cat","feature_światła-led__cat","feature_łopatki-zmiany-biegów__cat","seller_name__cat","feature_bluetooth__cat","param_marka-pojazdu__cat","feature_system-start-stop__cat","feature_światła-do-jazdy-dziennej__cat","param_kraj-pochodzenia__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:49:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10103.511310896827, 96.41132046670592)